## Day4-2 : database

In [4]:
import sqlite3
import time
import json

In [5]:
# initial local DB
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
cursor.execute('create table IF NOT EXISTS iot_table (time text NOT NULL, data text)')
cursor.close()
conn.commit()
conn.close()

In [14]:
# 寫入一筆資料

my_time = time.strftime("%Y-%m-%d %H:%M:%S")
my_data =  {"H": 80, "T": 32}

my_data = json.dumps(my_data, sort_keys=True)

#- insert into local DB
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
cursor.execute("insert into iot_table (time, data) values('%s', '%s')" %(my_time, my_data))
cursor.close()
conn.commit()
conn.close()

In [15]:
# 取出全部

conn = sqlite3.connect('test.db')
cursor = conn.cursor()
# select last 5 rows
res = cursor.execute('SELECT * FROM iot_table')
rows = res.fetchall()

print(rows)

[('2020-07-22 18:47:01', '{"H": 90, "T": 32}'), ('2020-07-22 18:47:29', '{"H": 90, "T": 32}'), ('2020-07-22 18:50:07', '{"H": 80, "T": 32}')]


In [16]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
# select last 5 rows
res = cursor.execute('SELECT * FROM iot_table ORDER BY time DESC LIMIT 1')
rows = res.fetchall()

print(rows)

[('2020-07-22 18:50:07', '{"H": 80, "T": 32}')]


In [ ]:
# remove rows

for row in rows:
    #print('delete:', row)
    res = cursor.execute("DELETE FROM iot_table WHERE time = '%s'" %(row[0]))
cursor.close()
conn.commit()
conn.close()

print('rows:', rows)

### 試著把空氣品質資料寫入DB中

### 排程: 把空氣品質資料寫入DB中

### 把資料讀出來，並繪成趨勢圖 (下午)

## Day4-2 : Firestore (Firebase)

### GCP 免費方案: https://cloud.google.com/free?hl=zh-tw

- firestore: https://cloud.google.com/firestore/docs/quickstart-servers?hl=zh-tw#python
- firebase: https://firebase.google.com/
    - 建議由這邊進入，開啟免費的base服務

### firebase

In [ ]:
#put

from firebase import firebase
import time

my_firebase = firebase.FirebaseApplication('https://fire-test-id.firebaseio.com', None)

my_time = time.strftime("%Y-%m-%d %H:%M:%S")
my_data = {'data': {'T': 25, 'H': 65}, 'time': my_time}
result = my_firebase.post(tag_name+'/log', my_data)
print('post result=', result)

result = my_firebase.put('', '/'+tag_name+'/rt', my_data)
print('put result=', result)

In [ ]:
#get

from firebase import firebase
import time

my_firebase = firebase.FirebaseApplication('https://fire-test-id.firebaseio.com', None)

result = my_firebase.get(tag_name, None)
result

### firestore (新的)
- 每日免費 50000 read, 20000 write, 20000 delete
- https://pypi.org/project/google-cloud/
- pip install google-cloud

In [2]:
from google.cloud import firestore

# Project ID is determined by the GCLOUD_PROJECT environment variable
db = firestore.Client()


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [3]:
from google.cloud import firestore

# Add a new document
db = firestore.Client()
doc_ref = db.collection(u'users').document(u'alovelace')
doc_ref.set({
    u'first': u'Ada',
    u'last': u'Lovelace',
    u'born': 1815
})


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [ ]:
# Then query for documents
users_ref = db.collection(u'users')

for doc in users_ref.stream():
    print(u'{} => {}'.format(doc.id, doc.to_dict()))